## 데이터 전처리

In [2]:
!pip install pandas

### 데이터 기본정보 확인

In [50]:
import pandas as pd

# CSV 파일 경로
file_path1 = '../data/관세청_HS부호_240101.csv'
file_path2 = '../data/통계청 국제표준산업분류 HSCODE 6단위 매핑.csv'
file_path3 = '../data/비식별된 해외기업별 영문 텍스트데이터.csv'

# CSV 파일 읽기
df1 = pd.read_csv(file_path1, converters={'HS부호':str, '성질통합분류코드':str})
df2 = pd.read_csv(file_path2, converters={'HS2017\n관세·통계통합품목분류(WCO, 한국)':str, 'ISIC4\n(국제표준산업분류, UN)':str, 'KSIC10\n한국표준산업분류':str})
df3 = pd.read_csv(file_path3, converters={'CODE':str})

# 데이터프레임 구조 확인
print("DataFrame 1: 관세청_HS부호_240101")
print(df1.info())
print(df1.head())

print("\nDataFrame 2: 통계청 국제표준산업분류 HSCODE 6단위 매핑")
print(df2.info())
print(df2.head())

print("\nDataFrame 3: 비식별된 해외기업별 영문 텍스트데이터")
print(df3.info())
print(df3.head())

# 데이터 분석
# 1. 관세청 HS부호 데이터 분석
hs_code_counts = df1['HS부호'].value_counts()
print("\nHS Code Counts in 관세청_HS부호_240101:")
print(hs_code_counts)

# 2. 통계청 HSCODE 6단위 매핑 데이터 분석
hs2017_mapping_counts = df2['HS2017\n관세·통계통합품목분류(WCO, 한국)'].value_counts()
print("\nHSCODE Mapping Counts in 통계청 국제표준산업분류 HSCODE 6단위 매핑:")
print(hs2017_mapping_counts)

# 3. 해외기업별 영문 텍스트 데이터 분석
# 텍스트 데이터에서 가장 많이 등장하는 단어 계산
word_counts = df3['DSC'].str.split(expand=True).stack().value_counts()
print("\nMost Common Words in 해외기업별 영문 텍스트데이터:")
print(word_counts.head(20))


DataFrame 1: 관세청_HS부호_240101
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12422 entries, 0 to 12421
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   HS부호       12422 non-null  object
 1   한글품목명      12422 non-null  object
 2   영문품목명      12422 non-null  object
 3   성질통합분류코드   12422 non-null  object
 4   성질통합분류코드명  11294 non-null  object
dtypes: object(5)
memory usage: 485.4+ KB
None
         HS부호   한글품목명              영문품목명  성질통합분류코드  성질통합분류코드명
0  0101211000  농가 사육용  For farm breeding  11020101        (말)
1  0101219000      기타              Other  11020101        (말)
2  0101291000     경주말  Horses for racing  11020101        (말)
3  0101299000      기타              Other  11020101        (말)
4  0101300000     당나귀              Asses  11020190  (기타 산 동물)

DataFrame 2: 통계청 국제표준산업분류 HSCODE 6단위 매핑
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8519 entries, 0 to 8518
Data columns (total 6 columns):
 #   Column             

In [15]:
# 분류 코드 = 분류 코드명
len(df1['성질통합분류코드명'].unique()) == len(df1['성질통합분류코드'].unique()) 

True

In [16]:
# 4개 차이 -> 영문만 사용
# 분류 코드에 비해 영문품목명이 더 세분화
len(df1['영문품목명'].unique())
len(df1['한글품목명'].unique())
#len(df1['성질통합분류코드'].unique()

6179

In [17]:
len(df1['HS부호'].unique())

12422

In [19]:
# HS CODE 6(국제)~10(한국) 단위
# 2 / 4 / 6으로 세분화, 6단위까지 실질적 분류
df1['HS부호_길이'] = df1['HS부호'].apply(len)

# 각 길이별 데이터 수 계산
hs_code_length_counts = df1['HS부호_길이'].value_counts().sort_index()
hs_code_length_counts

HS부호_길이
7        52
8       906
9       170
10    11294
Name: count, dtype: int64

In [20]:
df1['HS부호_6자리'] = df1['HS부호'].str[:6]

# 각 6자리별 데이터 수 계산
hs_code_6_digit_counts = df1['HS부호_6자리'].value_counts().sort_index()
hs_code_6_digit_counts


HS부호_6자리
010121    2
010129    2
010130    1
010190    1
010221    3
         ..
970529    1
970531    1
970539    1
970610    3
970690    3
Name: count, Length: 5613, dtype: int64

In [21]:
len(hs_code_6_digit_counts)

5613

In [24]:
first_col_df2 = df2.iloc[:, 0]
second_col_df3 = df3.iloc[:, 1]

# 공통 값 찾기
common_values = set(first_col_df2).intersection(set(second_col_df3))

# 공통 값의 개수와 일부 값 출력
common_values_count = len(common_values)
common_values_list = list(common_values)

common_values_count

241

In [25]:
print(len(df2['ISIC4\n(국제표준산업분류, UN)'].unique()))
print(len(df2['KSIC10\n한국표준산업분류'].unique()))
print(len(df2['HS2017\n관세·통계통합품목분류(WCO, 한국)'].unique()))

376
996
5359


### df2 ISIC - HS2017 매핑 및 df3 컬럼으로 붙임
df2 ISIC에 대응하는 HS2017 코드가 있어 매핑시켰고, ISIC 컬럼에 null 값이 많아 확인해보니 

ISIC 값이 null인 경우, HSCODE들이 바로 위 행의 ISIC 값에 대응된다는 것을 발견했습니다.

In [52]:
# Step 1: Remove rows with null values in all columns
df2_cleaned = df2.dropna(how='all')

# Step 2: 가장 가까운 이전 행 값으로 채우기 -> ffill 바로 위 값과 동일하게 변경
df2_cleaned[df2.columns[0]] = df2_cleaned[df2.columns[0]].fillna(method='ffill')

# HS2017 값 null이면 행 삭제
df2_cleaned = df2_cleaned.dropna(subset=[df2.columns[-2]])
df2_cleaned = df2_cleaned[df2_cleaned[df2.columns[-2]] != ""]

# Step 4: Create the dictionary with first column as key and list of second last column as value
mapping_dict_updated = df2_cleaned.groupby(df2_cleaned.columns[0])[df2_cleaned.columns[-2]].apply(list).to_dict()

# Remove duplicate values in the lists in divtionary
for key in mapping_dict_updated:
    mapping_dict_updated[key] = list(set(mapping_dict_updated[key]))

# Ensure keys with no corresponding values have an empty list
unique_keys = df2_cleaned[df2_cleaned.columns[0]].unique()
for key in unique_keys:
    if key not in mapping_dict_updated:
        mapping_dict_updated[key] = []

# Create a new column in df3 with the corresponding values from the updated dictionary
df3['Mapped_Values'] = df3['CODE'].map(mapping_dict_updated)


C:\Users\DILAB\AppData\Local\Temp\ipykernel_19336\4241613477.py:5: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df2_cleaned[df2.columns[0]] = df2_cleaned[df2.columns[0]].fillna(method='ffill')


In [53]:
df3['Mapped_Values']

0                                                     NaN
1       [010614, 041000, 010620, 030760, 010641, 50010...
2                                                     NaN
3                                                     NaN
4       [851220, 840991, 940110, 870821, 851150, 87081...
                              ...                        
9995                                             [491199]
9996                             [220210, 220299, 220110]
9997                                                  NaN
9998    [720610, 722240, 722230, 730511, 721810, 73059...
9999                                                  NaN
Name: Mapped_Values, Length: 10000, dtype: object

df3 기준 대략 3000개 데이터에 대해 매핑 되어 있고, 나머지 7000개는 매핑이 안되어 있습니다.

In [33]:
df3['Mapped_Values'].isnull().sum()

6988

In [34]:
len(df3['CODE'].unique())

273

### df1의 영문품목명 Other -> 성질통합분류코드명 번역으로 대체
df1의 영문품목명을 추후 자연어 학습에 쓰일 예정인데, Other 값이 많아 Other 값일 땐 성질통합분류코드명을 번역한 내용으로 채웠습니다.

In [54]:
!pip install sentencepiece
!pip install torch
!pip install transformers
!pip install huggingface-hub

In [38]:
import torch
from transformers import MarianMTModel, MarianTokenizer
from tqdm import tqdm

# Load the MarianMT model and tokenizer for Korean to English translation
model_name = 'Helsinki-NLP/opus-mt-ko-en'
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

# Move model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

device

c:\Users\DILAB\anaconda3\envs\public_data\Lib\site-packages\transformers\models\marian\tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


device(type='cuda')

In [55]:
torch.cuda.is_available()

True

In [56]:

# Function to translate and clean the '성질통합분류코드명' column
def translate_text(korean_text):
    # Tokenize the input text
    tokenized_text = tokenizer(korean_text, return_tensors='pt', padding=True, truncation=True).to(device)
    # Generate translation
    translated_tokens = model.generate(**tokenized_text)
    # Decode the translated tokens
    translated_text = tokenizer.decode(translated_tokens[0], skip_special_tokens=True)
    return translated_text

# Function to process each row
def process_row(row):
    if row['영문품목명'] == 'Other':
        korean_text = str(row['성질통합분류코드명']).strip('()')
        if '기타' in korean_text:
            korean_text = korean_text.replace('기타', '').strip()
        try:
            translated_text = translate_text(korean_text)
        except Exception as e:
            translated_text = 'Translation Error'
        row['영문품목명'] = translated_text
    return row

# Apply the function to the dataframe using tqdm for progress bar
tqdm.pandas()
df1 = df1.progress_apply(process_row, axis=1)

# Save the updated dataframe to a new CSV file
output_path_df1 = '../data/updated_관세청_HS부호_240101.csv'
df1.to_csv(output_path_df1, index=False)

print("Updated file saved to:", output_path_df1)



100%|██████████| 12422/12422 [02:28<00:00, 83.74it/s] 

Updated file saved to: ../data/updated_관세청_HS부호_240101.csv


In [60]:
df1 = pd.read_csv('../data/updated_관세청_HS부호_240101.csv', converters={'HS부호':str, '성질통합분류코드':str})
df1

,HS부호,한글품목명,영문품목명,성질통합분류코드,성질통합분류코드명
0,0101211000,농가 사육용,For farm breeding,11020101,(말)
1,0101219000,기타,HORSE,11020101,(말)
2,0101291000,경주말,Horses for racing,11020101,(말)
3,0101299000,기타,HORSE,11020101,(말)
4,0101300000,당나귀,Asses,11020190,(기타 산 동물)
...,...,...,...,...,...
12417,9706102000,악기류,Musical instruments,12050201,(수집품 및 골동품)
12418,9706103000,기타,Collecting and antiques,12050201,(수집품 및 골동품)
12419,9706901000,도자기류,Ceramics,12050201,(수집품 및 골동품)
12420,9706902000,악기류,Musical instruments,12050201,(수집품 및 골동품)


In [23]:
from multiprocessing import Pool, cpu_count

cpu_count()

24

### df3의 Mapped_Values(ISIC에 대응하는 HSCODE 6자리) -> 각각의 6자리에 대응되는 (HSCODE 10자리와 영문품목명)
6자리로 시작하는 10자리를 가지고 오고 그에 대응되는 영문품목명 또한 가져옵니다.

In [61]:
df3['Mapped_Values']

0                                                     NaN
1       [010614, 041000, 010620, 030760, 010641, 50010...
2                                                     NaN
3                                                     NaN
4       [851220, 840991, 940110, 870821, 851150, 87081...
                              ...                        
9995                                             [491199]
9996                             [220210, 220299, 220110]
9997                                                  NaN
9998    [720610, 722240, 722230, 730511, 721810, 73059...
9999                                                  NaN
Name: Mapped_Values, Length: 10000, dtype: object

In [62]:
for index, row in df3.iterrows():
    print(row.Mapped_Values)

nan
['010614', '041000', '010620', '030760', '010641', '500100', '010611', '010631', '010633', '010619', '010649', '040900']
nan
nan
['851220', '840991', '940110', '870821', '851150', '870810', '851120', '940130', '853190']
nan
nan
['731940', '732310', '731412', '860800', '731300', '831110', '848710', '830300', '741300', '860711', '732010', '830910', '731520', '731600', '830510', '850511', '731010', '731700', '830400', '731210', '732410', '731811']
['950611', '950691', '640212', '420321', '950631', '950710', '950621']
['846711', '846791', '846721']
nan
nan
nan
nan
nan
nan
['020725', '020742', '020450', '230110', '430110', '410210', '160220', '020760', '020321', '020711', '021091', '020724', '020850', '150290', '020610', '410320', '020311', '020751', '021011', '410390', '020752', '020500', '020430', '020810', '020712', '150190', '160100', '150110', '020410', '020741', '021020', '020910', '020680', '020713', '020210', '020110', '410120', '020630']
['040719', '010513', '010515', '010512',

In [63]:
# Function to find the corresponding hs code and 영문품목명
def find_corresponding_values(mapped_values):
    result = []
    if isinstance(mapped_values, list):
        for val in mapped_values:
            matched_rows = df1[df1['HS부호'].str[:len(val)] == val]
            for _, matched_row in matched_rows.iterrows():
                result.append((matched_row['HS부호'], matched_row['영문품목명']))
        return result
    else:
        return None  # If mapped_values is not a list or is null, return None

# Apply the function to df3 with tqdm progress bar
results = []
for index, row in tqdm(df3.iterrows(), total=len(df3)):
    results.append(find_corresponding_values(row.Mapped_Values))

df3['Corresponding_Values'] = results

# Save the updated df3 to a new CSV file
output_path_df3 = '../data/updated_비식별된_해외기업별_영문_텍스트데이터.csv'
df3.to_csv(output_path_df3, index=False)

print("Updated file saved to:", output_path_df3)

100%|██████████| 10000/10000 [01:07<00:00, 148.37it/s]

Updated file saved to: ../data/updated_비식별된_해외기업별_영문_텍스트데이터.csv


In [64]:
df3

,ID,CODE,DSC,Mapped_Values,Corresponding_Values
0,1,4520,"automotive repair shops, nec specialized auto...",NaN,None
1,2,0149,"general farms, primarily animals, nsk derives...","[010614, 041000, 010620, 030760, 010641, 50010...","[(0106141000, Pure-bred breeding animals), (01..."
2,3,4630,fish and seafoods the wholesale distribution ...,NaN,None
3,4,4510,"new and used car dealers, nsk manufactures a ...",NaN,None
4,4,2930,"automotive stampings, nsk manufacturing autom...","[851220, 840991, 940110, 870821, 851150, 87081...","[(85122010, Lighting equipment), (8512201010, ..."
...,...,...,...,...,...
9995,10641,1811,"commercial printing, nec commercial or job pr...",[491199],"[(4911990000, Printed)]"
9996,10642,1104,"bottled and canned soft drinks, nsk manufactu...","[220210, 220299, 220110]","[(2202101000, Coloured), (2202109000, Drinks),..."
9997,10643,4659,"professional equipment, nec, nsk the wholesal...",NaN,None
9998,10644,2410,"blast furnaces and steel mills, nsk manufactu...","[720610, 722240, 722230, 730511, 721810, 73059...","[(7206100000, Ingots), (7222400000, Angles, sh..."


### 영문품목명을 CLS 토큰으로 전부 합친다.
DSC값과 영문품목명 set이 유사하다는 것을 학습에 사용하기 위헤 CLS 토큰으로 합쳐 하나의 문장으로 만듭니다.

In [65]:
# Create a new column with the CLS token joined string of the second element in the tuples
def create_joined_string(corresponding_values):
    if corresponding_values:
        return ' [CLS] '.join([tup[1] for tup in corresponding_values if isinstance(tup, tuple) and len(tup) > 1])
    else:
        return ''

df3['Joined_Corresponding_Values'] = df3['Corresponding_Values'].apply(create_joined_string)

In [66]:
df3["Joined_Corresponding_Values"]

0                                                        
1       Pure-bred breeding animals [CLS] Living Animal...
2                                                        
3                                                        
4       Lighting equipment [CLS] Of light-emitting dio...
                              ...                        
9995                                              Printed
9996    Coloured [CLS] Drinks [CLS] Beverage based on ...
9997                                                     
9998    Ingots [CLS] Angles, shapes and sections [CLS]...
9999                                                     
Name: Joined_Corresponding_Values, Length: 10000, dtype: object

In [67]:
output_path_df3 = '../data/updated_비식별된_해외기업별_영문_텍스트데이터.csv'
df3.to_csv(output_path_df3, index=False)

### HSCDOE 6자리에 대응되는 영문 text가 없음 -> 외부 데이터 사용
영문품목명이 존재하는 HSCODE 10자리의 영문품목명만을 가져와서 학습할 계획이었으나,

학습한 데이터를 토대로 추론을 하는 것이 논리적이지 않다고 판단했고, 매핑되지 않은 7000개에 대한 데이터는 동일한 방법으로 학습할 수가 없기 때문에 HSCODE 6자리에 대한 영문 품목명을 외부 데이터를 통해 가지고 와 학습에 사용하기로 했습니다.

HSCODE 6자리에 대한 한글 품목명은 있지만, 이를 번역하는 것보다 외부 데이터를 활용 하는 것이 더 안정적이라 생각해 이를 사용했습니다.

In [68]:
import requests

url = 'https://comtradeapi.un.org/files/v1/app/reference/H5.json'
data = requests.get(url).json()

hs6_list = [item['text'] for item in data['results'] if len(item['id']) == 6]
hs6_info = [(item.split(" - ", 1)[0], item.split(" - ", 1)[1]) for item in hs6_list]

hs6_info

[('010121', 'Horses; live, pure-bred breeding animals'),
 ('010129', 'Horses; live, other than pure-bred breeding animals'),
 ('010130', 'Asses; live'),
 ('010190', 'Mules and hinnies; live'),
 ('010221', 'Cattle; live, pure-bred breeding animals'),
 ('010229', 'Cattle; live, other than pure-bred breeding animals'),
 ('010231', 'Buffalo; live, pure-bred breeding animals'),
 ('010239', 'Buffalo; live, other than pure-bred breeding animals'),
 ('010290', 'Bovine animals; live, other than cattle and buffalo'),
 ('010310', 'Swine; live, pure-bred breeding animals'),
 ('010391',
  'Swine; live, other than pure-bred breeding animals, weighing less than 50kg'),
 ('010392',
  'Swine; live, other than pure-bred breeding animals, weighing 50kg or more'),
 ('010410', 'Sheep; live'),
 ('010420', 'Goats; live'),
 ('010511',
  'Poultry; live, fowls of the species Gallus domesticus, weighing not more than 185g'),
 ('010512', 'Poultry; live, turkeys, weighing not more than 185g'),
 ('010513', 'Poultry

In [74]:
df3['Mapped_Values']


0                                                     NaN
1       [010614, 041000, 010620, 030760, 010641, 50010...
2                                                     NaN
3                                                     NaN
4       [851220, 840991, 940110, 870821, 851150, 87081...
                              ...                        
9995                                             [491199]
9996                             [220210, 220299, 220110]
9997                                                  NaN
9998    [720610, 722240, 722230, 730511, 721810, 73059...
9999                                                  NaN
Name: Mapped_Values, Length: 10000, dtype: object

In [77]:
# hs6_dict 생성
hs6_dict = {code: text for code, text in hs6_info}

# 새로운 컬럼에 대응되는 텍스트 리스트 추가
def get_text_list(mapped_values):
    if isinstance(mapped_values, list) and mapped_values:  # 리스트이고 비어 있지 않은 경우에만 처리
        texts = [hs6_dict[code] for code in mapped_values if code in hs6_dict]
        return [text for text in texts if text is not None]
    return []

df3['hs6_text'] = df3['Mapped_Values'].apply(get_text_list)

# 결과 출력
print(df3[['Mapped_Values', 'hs6_text']])


                                          Mapped_Values  \
0                                                   NaN   
1     [010614, 041000, 010620, 030760, 010641, 50010...   
2                                                   NaN   
3                                                   NaN   
4     [851220, 840991, 940110, 870821, 851150, 87081...   
...                                                 ...   
9995                                           [491199]   
9996                           [220210, 220299, 220110]   
9997                                                NaN   
9998  [720610, 722240, 722230, 730511, 721810, 73059...   
9999                                                NaN   

                                               hs6_text  
0                                                    []  
1     [Mammals; live, rabbits and hares, Animal prod...  
2                                                    []  
3                                                    []  
4

In [84]:
df3["Mapped_Values"]

0                                                     NaN
1       [010614, 041000, 010620, 030760, 010641, 50010...
2                                                     NaN
3                                                     NaN
4       [851220, 840991, 940110, 870821, 851150, 87081...
                              ...                        
9995                                             [491199]
9996                             [220210, 220299, 220110]
9997                                                  NaN
9998    [720610, 722240, 722230, 730511, 721810, 73059...
9999                                                  NaN
Name: Mapped_Values, Length: 10000, dtype: object

HSCODE 6자리에 대한 영문품목명을 아까와 같이 CLS 토큰으로 이어 하나의 문장으로 만들어줍니다.

In [79]:
# Create a new column with the CLS token joined string of the second element in the tuples
def create_joined_string(corresponding_values):
    if corresponding_values:
        return ' [CLS] '.join(corresponding_values)
    else:
        return ''

df3['joined_hs6_text'] = df3['hs6_text'].apply(create_joined_string)

In [82]:
df3['joined_hs6_text'][1]

'Mammals; live, rabbits and hares [CLS] Animal products; edible, n.e.c. in this or other chapters [CLS] Reptiles; live (including snakes and turtles) [CLS] Molluscs; snails, other than sea snails, whether in shell or not, live, fresh, chilled, frozen, dried, salted, in brine, or smoked, cooked or not before or during the smoking process [CLS] Insects; live, bees [CLS] Silk; silk-worm cocoons suitable for reeling [CLS] Mammals; live, primates [CLS] Birds; live, birds of prey [CLS] Birds; live, ostriches; emus (Dromaius novaehollandiae) [CLS] Mammals; live, other than primates, whales, dolphins, porpoises (mammals of the order Cetacea); manatees, dugongs (mammals of the order Sirenia); seals, sea lions, walruses (mammals of the suborder Pinnipedia), camels, other camelids, rabbits and hares [CLS] Insects; live, other than bees [CLS] Honey; natural'

In [83]:
df3.to_csv(output_path_df3, index=False)

## 위 내용은 데이터만으로 매핑된 3000개에 대한 전처리 단계입니다.

### HSCODE_6자리 매핑

1. 기업 DSC 인코딩
2. 외부 HS2017 데이터 인코딩
3. HSCODE 2자리 단위씩 반복 매핑

### 기업 DSC 임베딩 불러오기
파인튜닝 시킨 모델로 임베딩 추출 후 불러오기